In [ ]:
import sklearn.preprocessing
import pydataset

In [ ]:
df = pydataset.data('sat.act')

In [ ]:
df.head()

In [ ]:
# 1. Make the thing
scaler = sklearn.preprocessing.MinMaxScaler()

# 2. Fit the thing
scaler.fit(df[['age']])

# 3. Use the thing
df['age_scaled'] = scaler.transform(df[['age']])

In [ ]:
df.head()

In [ ]:
# How to transform multiple cols

df_scaled = df.copy()


scaler = sklearn.preprocessing.MinMaxScaler()


scaler.fit(df[['age', 'ACT', 'SATV', 'SATQ']])
df_scaled[['age', 'ACT', 'SATV', 'SATQ']] = scaler.transform(df[['age', 'ACT', 'SATV', 'SATQ']])

In [ ]:
df_scaled.head()

In [ ]:
scaler = sklearn.preprocessing.MinMaxScaler()

# Split and Scale Exercises

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from math import sqrt
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, QuantileTransformer, PowerTransformer, RobustScaler, MinMaxScaler
from pydataset import data

from env import host, user, password
from wrangle import wrangle_telco

> As a customer analyst, I want to know who has spent the most money with us over their lifetime. I have monthly charges and tenure, so I think I will be able to use those two attributes as features to estimate total_charges. I need to do this within an average of $5.00 per customer.

### Create split_scale.py that will contain the functions that follow. Each scaler function should create the object, fit and transform both train and test. They should return the scaler, train dataframe scaled, test dataframe scaled. Be sure your indices represent the original indices from train/test, as those represent the indices from the original dataframe. Be sure to set a random state where applicable for reproducibility!

1. split_my_data(X, y, train_pct)
1. standard_scaler()
1. scale_inverse()
1. uniform_scaler()
1. gaussian_scaler()
1. min_max_scaler()
1. iqr_robust_scaler()

In [2]:
telco = wrangle_telco()

In [3]:
telco.head()

,customer_id,monthly_charges,tenure,total_charges
0,0013-SMEOE,109.70,71,7904.25
1,0014-BMAQU,84.65,63,5377.80
2,0016-QLJIS,90.45,65,5957.90
3,0017-DINOC,45.20,54,2460.55
4,0017-IUDMW,116.80,72,8456.75


In [73]:
telco.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1695 entries, 0 to 1694
Data columns (total 4 columns):
customer_id        1695 non-null object
monthly_charges    1695 non-null float64
tenure             1695 non-null int64
total_charges      1695 non-null float64
dtypes: float64(2), int64(1), object(1)
memory usage: 66.2+ KB


In [ ]:
# X = telco[['monthly_charges','tenure']]
# y = telco[['total_charges']]

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=19)

In [ ]:
# print('X_train:', X_train.shape)
# print('X_test:', X_test.shape)
# print('y_train:', y_train.shape)
# print('y_test:', y_test.shape)

# assert X_train.shape[0] == y_train.shape[0]
# assert X_test.shape[0] == y_test.shape[0]

In [ ]:
# X_train.head()

In [43]:
def split_my_data(X, y, train_pct):
    """
    Take in X_train, X_test, y_train, y_test
    """
    return (train_test_split(X, y, train_size = train_pct))

In [44]:
X = telco[['monthly_charges','tenure']]
y = telco[['total_charges']]

X_train, X_test, y_train, y_test = split_my_data(X, y, .8)

In [45]:
print('X_train:', X_train.shape)
print('X_test:', X_test.shape)
print('y_train:', y_train.shape)
print('y_test:', y_test.shape)

assert X_train.shape[0] == y_train.shape[0]
assert X_test.shape[0] == y_test.shape[0]

X_train: (1356, 2)
X_test: (339, 2)
y_train: (1356, 1)
y_test: (339, 1)


In [46]:
X_train

,monthly_charges,tenure
524,25.35,0
37,105.65,67
878,79.15,43
1476,77.55,71
51,108.90,46
...,...,...
1544,26.10,10
337,78.95,72
326,102.60,63
1531,108.20,72


In [47]:
def standard_scaler(train, test):
    """
    When calling the function rename the variables to maintain original:
    scaler, X_train_scaled, X_test_scaled = standard_scaler(X_train, X_test)
    """
    scaler = StandardScaler(copy=True, with_mean=True, with_std=True).fit(train)
    train_scaled = pd.DataFrame(scaler.transform(train), columns=train.columns.values).set_index([train.index.values])
    test_scaled = pd.DataFrame(scaler.transform(test), columns=test.columns.values).set_index([test.index.values])
    return scaler, train_scaled, test_scaled

In [48]:
scaler, X_train_scaled, X_test_scaled = standard_scaler(X_train, X_test)

In [49]:
X_train_scaled

,monthly_charges,tenure
524,-1.033950,-3.199128
37,1.284921,0.562565
878,0.519665,-0.784907
1476,0.473461,0.787143
51,1.378773,-0.616473
...,...,...
1544,-1.012292,-2.637682
337,0.513889,0.843288
326,1.196845,0.337986
1531,1.358559,0.843288


In [54]:
def scale_inverse(train_scaled, test_scaled, scaler):
    # If we wanted to return to original values:
    # apply to train
    train_unscaled = pd.DataFrame(scaler.inverse_transform(train_scaled), columns=train_scaled.columns.values).set_index([train_scaled.index.values])
    # apply to test
    test_unscaled = pd.DataFrame(scaler.inverse_transform(test_scaled), columns=test_scaled.columns.values).set_index([test_scaled.index.values])
    return train_unscaled, test_unscaled

In [58]:
X_train_unscaled, X_test_unscaled = scale_inverse(X_train_scaled, X_test_scaled, standard_scaler)

In [59]:
X_train_unscaled

,monthly_charges,tenure
524,25.35,0.0
37,105.65,67.0
878,79.15,43.0
1476,77.55,71.0
51,108.90,46.0
...,...,...
1544,26.10,10.0
337,78.95,72.0
326,102.60,63.0
1531,108.20,72.0


In [62]:
def uniform_scaler(train, test):
    scaler = QuantileTransformer(n_quantiles=100, output_distribution='uniform', random_state=123, copy=True).fit(train)
    train_scaled = pd.DataFrame(scaler.transform(train), columns=train.columns.values).set_index([train.index.values])
    test_scaled = pd.DataFrame(scaler.transform(test), columns=test.columns.values).set_index([test.index.values])
    return train_scaled, test_scaled

In [65]:
X_train_uniform_scaled, X_test_uniform_scaled = uniform_scaler(X_train, X_test)

In [66]:
X_train_uniform_scaled

,monthly_charges,tenure
524,0.329966,0.000000
37,0.868342,0.575758
878,0.605728,0.191919
1476,0.595276,0.762626
51,0.894465,0.217172
...,...,...
1544,0.363751,0.026936
337,0.604398,1.000000
326,0.833083,0.469697
1531,0.887012,1.000000


In [68]:
def gaussian_scaler(train, test):
    scaler = PowerTransformer(method='yeo-johnson', standardize=False, copy=True).fit(train)
    train_scaled = pd.DataFrame(scaler.transform(train), columns=train.columns.values).set_index([train.index.values])
    test_scaled = pd.DataFrame(scaler.transform(test), columns=test.columns.values).set_index([test.index.values])
    return train_scaled, test_scaled

In [69]:
X_train_gaussian_scaled, X_test_gaussian_scaled = gaussian_scaler(X_train, X_test)

In [70]:
X_train_gaussian_scaled

,monthly_charges,tenure
524,6.767148,0.000000
37,13.731171,5808.421675
878,11.974793,2184.835879
1476,11.858248,6604.062342
51,13.927701,2533.743211
...,...,...
1544,6.871981,96.694224
337,11.960302,6811.853002
326,13.543446,5069.014413
1531,13.885668,6811.853002


In [71]:
def min_max_scaler(train, test):
    scaler = MinMaxScaler(copy=True, feature_range=(0,1)).fit(train)
    train_scaled = pd.DataFrame(scaler.transform(train), columns=train.columns.values).set_index([train.index.values])
    test_scaled = pd.DataFrame(scaler.transform(test), columns=test.columns.values).set_index([test.index.values])
    return train_scaled, test_scaled

In [72]:
def iqr_robust_scaler(train, test):
    scaler = RobustScaler(quantile_range=(25.0,75.0), copy=True, with_centering=True, with_scaling=True).fit(train)
    train_scaled = pd.DataFrame(scaler.transform(train), columns=train.columns.values).set_index([train.index.values])
    test_scaled = pd.DataFrame(scaler.transform(test), columns=test.columns.values).set_index([test.index.values])
    return train_scaled, test_scaled